# SCAFFOLD for Non-IID Setting 

## Introduction

In this notebook we implement the SCAFFOLD algorithm, as proposed by Karimireddy et al. in the paper [SCAFFOLD: Stochastic Controlled Averaging for Federated Learning](https://proceedings.mlr.press/v119/karimireddy20a/karimireddy20a.pdf) in 2020. The general idea is to introduce control variates to the local model updates, in order to mitigate the effects that data heterogeneity has on the learning process. 

In [1]:
from collections import OrderedDict
from typing import List, Tuple, Optional, Union, Dict
import os
from pathlib import Path
from logging import WARNING, WARN, INFO, DEBUG

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
from torch.optim import SGD
from datasets.utils.logging import disable_progress_bar

import flwr
from flwr.client import Client, ClientApp, NumPyClient
from flwr.common import (
    EvaluateIns,
    EvaluateRes,
    FitIns,
    FitRes,
    Parameters,
    Context, 
    Code,
    Scalar, 
    ndarrays_to_parameters,
    parameters_to_ndarrays,
    )
from flwr.common.typing import GetParametersIns
from flwr.common.logger import log
from flwr.server import ServerApp, ServerConfig, ServerAppComponents, Server
from flwr.server.strategy import Strategy, FedAvg
from flwr.server.strategy.aggregate import aggregate, weighted_loss_avg
from flwr.server.client_proxy import ClientProxy
from flwr.server.client_manager import ClientManager, SimpleClientManager
from flwr.server.server import FitResultsAndFailures, fit_clients
from flwr.simulation import run_simulation

Next, we need to make some imports from our own package. We will import the ```SmallCNN``` neural net object, as this is what we are running all the tests with. We will also import the function ```load_datasets```, which returns a tuple of ```(trainloader,valloader,testloader)```. We will again make use of the ```test``` function, which was introduced in the previous notebook, as well as some utility functions ```get_parameters``` and ```set_parameters```. Feel free to revisit the code, or you can simply use the python function ```help``` (eg ```help(test)```).

In [2]:
from fedlearn.model import SmallCNN, test
from fedlearn.data_loader import load_datasets
from fedlearn.utils import set_parameters, get_parameters

In [3]:
help(load_datasets)

Help on function load_datasets in module fedlearn.data_loader:

load_datasets(partition_id: int, partition_method: str, partitioner_kwargs: Dict[str, Union[str, int, float]], batch_size: int = 64, cache_dir: str = 'data') -> Tuple[torch.utils.data.dataloader.DataLoader, torch.utils.data.dataloader.DataLoader, torch.utils.data.dataloader.DataLoader]
    function for loading CIFAR-10 dataset and partitioning it for federated learning.
    
    Parameters:
        partition_id:       int, the ID of the partition to load.
        partition_method:   str, the method to use for partitioning the dataset.
        partitioner_kwargs: Dict[str, Union[str, int, float]], the parameters for the partitioner.
        batch_size:         int, the size of each batch for training and validation.
        cache_dir:          str, the directory to cache the dataset.
        
    Returns:
        Tuple[DataLoader, DataLoader, DataLoader]: trainloader, valloader, and testloader.



In [4]:
DATADIR = Path().cwd().parent / "data" / "flower_dataset"       # specify your data directory
LOGDIR = Path().cwd().parent / "logs" / "scaffold"              # specify your log directory

# creates the log directory if it does not exist
if not LOGDIR.exists():
    LOGDIR.mkdir(parents=True, exist_ok=True)

In [5]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu" # if running on mac, use: "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Training on {DEVICE}")
print(f"Flower {flwr.__version__} / PyTorch {torch.__version__}")
disable_progress_bar()

Training on cuda
Flower 1.18.0 / PyTorch 2.7.0+cu126


We are now ready to specify some simulation parameters. We arbitrarily set the number of clients/partitions to 10, specify a batch size of 64, and choose the dirichlet method for partitioning the data, with $\alpha=0.5$. For other data partitioning schemes, we encourage you to have a look at the ```flwr_datasets``` documentation on partitioners [here](https://flower.ai/docs/datasets/ref-api/flwr_datasets.partitioner.html).

In [6]:
NUM_PARTITIONS = 10 # Number of partitions for the federated dataset same as the number of clients
BATCH_SIZE = 64
PARTITION_METHOD = "dirichlet"  # Options: "iid", "dirichlet", "shard"
PARTITIONER_KWARGS = {
    "num_partitions": NUM_PARTITIONS,   # Number of partitions to create
    "alpha": 0.5,                       # Dirichlet parameter, only used if partition_method is "dirichlet"
    "partition_by": "label"             # Partition by label, only used if partition_method is "dirichlet" or "shard"
}

# Load datasets for partition 0 to check if everything works
_, _, _ = load_datasets(
    partition_id=0,                         # specify partition ID to load
    partition_method=PARTITION_METHOD,      # Method to partition the dataset
    partitioner_kwargs=PARTITIONER_KWARGS,  # Parameters for the partitioner
    batch_size=BATCH_SIZE,                  # Batch size for the DataLoader    
    cache_dir=DATADIR                       # Directory to cache the datasets
)

Using the latest cached version of the dataset since cifar10 couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'plain_text' at c:\Users\Phill\Documents\School\MSc\3. Sem\Federated Learning\FedLearning-Final\data\flower_dataset\cifar10\plain_text\0.0.0\0b2714987fa478483af9968de7c934580d0bb9a2 (last modified on Wed Jun  4 13:48:33 2025).


### Define Scaffold Optimizer

Recall that the local update in Scaffold is given by

$$
w^{(i)} \gets w^{(i)} - \eta_l \left( g_i(w^{(i)}) + c - c_i \right)
$$

Which can be seen as a gradient correction to Stochastic Gradient Descent (SGD). We may therefore extend the pytorch ```SGD``` class. We do this by computing the the regular SGD step, then adding the correction manually:

$$
\begin{align*}
w^{(i)} &\gets w^{(i)} - \eta_l \, g_i\left(w^{(i)}\right) \\
w^{(i)} &\gets w^{(i)} - \eta_l (c - c_i)
\end{align*}
$$

In [7]:
class ScaffoldOptimizer(SGD):
    """
    Extension of the SGD optimizer for the Scaffold algorithm.
    This optimizer applies a correction term based on the global 
    and client control variables. 
    """
    def __init__(self, params, lr, momentum=0., weight_decay=0.):
        super().__init__(params, lr, momentum, weight_decay)

    def step_custom(self, global_cv, client_cv):
        """
        Perform a single optimization step.
        :param global_cv: Global control variable
        :param client_cv: Client control variable
        """
        # compute regular SGD step
        #   w <- w - lr * grad
        super().step() 

        # now add the correction term
        #   w <- w - lr * (g_cv - c_cv)
        device = self.param_groups[0]["params"][0].device
        for group in self.param_groups:
            for param, g_cv, c_cv in zip(group["params"], global_cv, client_cv):
                # here we add the correction term to each parameter tensor.
                # the alpha value scales the correction term
                    g_cv, c_cv = g_cv.to(device), c_cv.to(device)
                    param.data.add_(g_cv - c_cv, alpha=-group["lr"]) 

As in the previous notebook, we need to implement a ```train``` function that performs the local update for a client. As the control variates are needed for the correction step, we cannot simply reuse the function from the previous notebook. Therefore, we need to implement a similar function, that takes the control variates into account.

In [8]:
def train_scaffold(net: torch.nn.Module, 
                   device: torch.device, 
                   trainloader: torch.utils.data.DataLoader,
                   criterion: nn.Module,
                   num_epochs: int, 
                   lr: float, 
                   momentum: float, 
                   weight_decay: float, 
                   global_cv: List[torch.Tensor], 
                   client_cv: List[torch.Tensor],
                   ) -> None:
    """
    Function that trains a model using the Scaffold optimization algorithm.
    Parameters:
        net:            The neural network model to train.
        device:         The device to run the training on (CPU or GPU).
        trainloader:    DataLoader for the training data.
        criterion:      Loss function to use for training.
        num_epochs:     Number of epochs to train the model.
        lr:             Learning rate for the optimizer.
        momentum:       Momentum factor for the optimizer.
        weight_decay:   Weight decay (L2 penalty) for the optimizer.
        global_cv:      Global control variables for Scaffold.
        client_cv:      Client control variables for Scaffold.
    """
    net.to(device)
    net.train()

    # initialize the custom Scaffold optimizer
    optimizer = ScaffoldOptimizer(
        net.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay
    )
    
    for _ in range(num_epochs):
        for batch in trainloader:
            Xtrain, Ytrain = batch["img"].to(device), batch["label"].to(device)
            optimizer.zero_grad()
            output = net(Xtrain)
            loss = criterion(output, Ytrain)

            loss.backward()
            
            # Perform a single optimization step with the control variables
            optimizer.step_custom(global_cv, client_cv)

Now we get to the difficult part. As is the previous notebook, we need to define a custom client class for the flower framework. However, as the algorithm requires aggregation and storage of a global control variate as well, we must also implement both the aggregation class, called a ```Strategy```, as well as the server class, called a ```Server```, from scratch.

We will start with the client class, as we are most familiar with this. Below are some important considerations:

1. We can inherit the  ```NumPyClient``` class from the flower framework, however we need to remember to convert between ```np.ndarray``` and ```torch.tensor``` before and after local updates.
2. There are 3 methods we need to implement, with a predefined structure
   1. ```fit()```
   2. ```get_parameters()```
   3. ```evaluate()```

The ```fit()``` method takes 2 parameters, ```parameters```, which in our case will be the concatenated global model parameters and the global control variate, and ```config```, which we will ignore for now. This method is responsible for all client-side updates, which means that we will call the ```train_scaffold``` function inside it to perform the local model update, as well as computing the updated local control variate. We will update the local control variate according to Option II from the original paper, which is given by:

$$
\begin{equation}
c_i^+ \gets c_i - c + \frac{1}{K \eta_l}(w_t - w_{t+1}^{(i)}) 
\end{equation}
$$

Where $K$ is the number of of descent steps taken, ie number of epochs times the number of mini-batches. 

We also need to specify what exactly is returned by the method. For clarity and in order to remain close in notation to the original paper, we will return the difference between the input model parameters and the locally updated parameters (returned from ```train_scaffold```), concatenated with the difference between the new and previous local control variate. Concretely, we return

$$
[w_{t+1}^{(i)} - w_t,\,\, c_i^+ - c_i]
$$

This is due to how the server-side updates look:

$$
\begin{align}
   w_{t+1} &\gets w_t + \frac{\eta_g}{|\mathcal{S}|} \sum_{i \in \mathcal{S}} \frac{n^{(i)}}{n}(w_{t+1}^{(i)} - w_t) \\
   c &\gets c + \frac{1}{N} \sum_{i \in \mathcal{S}} \frac{n^{(i)}}{n}(c_i^+ - c_i)
\end{align}
$$

In [9]:
class ScaffoldClient(NumPyClient):
    def __init__(self, 
                 partition_id: int, 
                 net: torch.nn.Module, 
                 trainloader: torch.utils.data.DataLoader, 
                 valloader: torch.utils.data.DataLoader,
                 criterion: nn.Module,
                 device: torch.device,
                 num_epochs: int,
                 lr: float,
                 momentum: float,
                 weight_decay: float,
                 save_dir: Optional[str] = None,
                 ):
        self.partition_id = partition_id
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader
        self.criterion = criterion
        self.device = device
        self.num_epochs = num_epochs
        self.lr = lr
        self.momentum = momentum
        self.weight_decay = weight_decay

        # define directory to save client control variates
        if save_dir is None:
            save_dir = "client_cvs"

        # create directory if it does not exist
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)

        # define the path to save the client control variates
        self.save_name = os.path.join(save_dir, f"client_{self.partition_id}_cv.pt")

        # initialize client control variates
        self.client_cv = [torch.zeros(param.shape).to(torch.float32) for param in self.net.state_dict().values()]


    # Here is where all the training logic and control variate updates happen
    def fit(self, parameters: List[np.ndarray], config: dict) -> Tuple[List[np.ndarray], int, dict]:

        # the global parameters are packed together with the global control variates
        # in the form [params, global_cv]. we start by separating them
        params = parameters[:len(parameters) // 2]          # list of np.ndarray
        global_cv = parameters[len(parameters) // 2:]       # list of np.ndarray

        # load the current global model:
        set_parameters(self.net, params)

        # load client control variates, if they exist:
        if os.path.exists(self.save_name):
            self.client_cv = torch.load(self.save_name)     # list of torch.tensor

        # convert global control variates to tensors
        global_cv_torch = [torch.tensor(cv).to(torch.float32) for cv in global_cv]  # list of torch.tensor

        # call the training function
        train_scaffold(
            net=self.net,
            device=self.device,
            trainloader=self.trainloader,
            criterion=self.criterion,
            num_epochs=self.num_epochs,
            lr=self.lr,
            momentum=self.momentum,
            weight_decay=self.weight_decay,
            global_cv=global_cv_torch,          # passing list of torch.tensor
            client_cv=self.client_cv            # passing list of torch.tensor
        )

        # update the client control variates
        yi = get_parameters(self.net)           # list of np.ndarray

        # compute coefficient for the control variates
        # 1 / (K * eta) where K is the number of backward passes (num_epochs * len(trainloader))
        coeff = 1. / (self.num_epochs * len(self.trainloader) * self.lr) 

        client_cv = [cv.numpy() for cv in self.client_cv]  # list of np.ndarray

        # define new list for udated client control variates
        client_cv_new = []

        # compute client control variate update according to eq (1), list of np.ndarray
        for xj, yj, cj, cij in zip(params, yi, global_cv, client_cv):
            client_cv_new.append(
                cij - cj + coeff * (xj - yj)
            ) 

        # compute server updates
        server_update_x = [yj - xj for xj, yj in zip(params, yi)]
        server_update_c = [cij_n - cij for cij_n, cij in zip(client_cv_new, client_cv)]

        # convert client cvs back to torch tensors
        self.client_cv = [torch.tensor(cv).to(torch.float32) for cv in client_cv_new]  

        # save the updated client control variates
        torch.save(self.client_cv, self.save_name)

        #concatenate server updates
        server_update = server_update_x + server_update_c

        return server_update, len(self.trainloader.dataset), {}


    def get_parameters(self, config: dict) -> List[np.ndarray]:
        return get_parameters(self.net)


    def evaluate(self, parameters: List[np.ndarray], config: dict) -> Tuple[float, int, dict]:
        set_parameters(self.net, parameters)
        avg_loss, accuracy = test(
            net=self.net,
            device=self.device,
            testloader=self.valloader,
            criterion=self.criterion
        )
        return float(avg_loss), len(self.valloader), {"accuracy": accuracy}

Now that we have the flower client defined, we need to define a constructor function which the flower framework can use to instatiate clients as it goes. This is done similarly to in the previous notebook. It should be noted that all the simulation parameters, such as ```partition_method```, ```batch_size``` etc. can be passed to the ```client_fn``` function thru the context. This is useful when simulations are run from the command line. For now, we will ignore this, and hardcode it into the function.

In [10]:
def client_fn(context: Context) -> Client:
    partition_id = context.node_config["partition-id"]
    num_partitions = context.node_config["num-partitions"]

    partitioner_kwargs = {
        "num_partitions": num_partitions,   # Number of partitions to create
        "alpha": 0.5,                       # Dirichlet parameter
        "partition_by": "label"             # Partition by label
    }

    trainloader, valloader, _ = load_datasets(
        partition_id=partition_id,              # specify partition ID to load
        partition_method=PARTITION_METHOD,      # Method to partition the dataset
        partitioner_kwargs=partitioner_kwargs,  # Parameters for the partitioner
        batch_size=BATCH_SIZE,                  # Batch size for the DataLoader    
        cache_dir=DATADIR                       # Directory to cache the datasets
    )

    net = SmallCNN().to(DEVICE)
    criterion = nn.CrossEntropyLoss()
    
    # Define hyperparameters for training
    num_epochs = 5
    lr = 1e-2
    momentum = 0.
    weight_decay = 0.

    return ScaffoldClient(
        partition_id=partition_id,
        net=net,
        trainloader=trainloader,
        valloader=valloader,
        criterion=criterion,
        device=DEVICE,
        num_epochs=num_epochs,
        lr=lr,
        momentum=momentum,
        weight_decay=weight_decay,
        save_dir=LOGDIR / "client_cvs"
    ).to_client()


client = ClientApp(client_fn=client_fn)

Now we are ready to define our strategy. This looks complex on the surface, however the really key component here is the ```aggregate_fit()``` method. This method expects 3 input parameters:

1. server_round: This parameter is not important to us right now.
2. results: This is a list of tuples, containing ```(ClientProxy,FitRes)```. We are only interested in the ```FitRes```, which presumably is short for "Fit results". This is a class that contains 2 attributes that we are interested in: ```parameters```, which contains the concatenated model parameters and local control variates, as returned by the ```fit()``` method of the ```ScaffoldClient```, and ```num_examples```, which contains how many datapoints the client has. 
3. failures: similar to results in datatype, but can also contain exceptions in the list. As we are concerned with simulations where we expect all clients to behave regularly, this can be ignored for now.

In this method, we are only concerned with "aggregating" the results, and not the full global update (confusing I know). This amounts to computing purely the sum portion of the global updates (both model parameters and global control variate). The server will be responsible for the full update.


In [11]:
class ScaffoldStrategy(Strategy):
    def __init__(
        self,
        total_num_clients: int,
        fraction_fit: float = 1.0,
        fraction_evaluate: float = 1.0,
        min_fit_clients: int = 2,
        min_evaluate_clients: int = 2,
        min_available_clients: int = 2,
        evaluate_fn: Optional[callable] = None,
        accept_failures: bool = False,
        fit_metrics_aggregation_fn: Optional[callable] = None,
    ) -> None:
        super().__init__()
        self.total_num_clients = total_num_clients
        self.fraction_fit = fraction_fit
        self.fraction_evaluate = fraction_evaluate
        self.min_fit_clients = min_fit_clients
        self.min_evaluate_clients = min_evaluate_clients
        self.min_available_clients = min_available_clients
        self.evaluate_fn = evaluate_fn
        self.accept_failures = accept_failures
        self.fit_metrics_aggregation_fn = fit_metrics_aggregation_fn


    def __repr__(self) -> str:
        return "ScaffoldStrategy"


    def initialize_parameters(
        self, 
        client_manager: ClientManager
    ) -> Optional[Parameters]:
        """Initialize global model parameters."""
        net = SmallCNN()
        parameters = get_parameters(net)        
        return ndarrays_to_parameters(parameters)


    def configure_fit(
        self, 
        server_round: int, 
        parameters: Parameters, 
        client_manager: ClientManager
    ) -> List[Tuple[ClientProxy, FitIns]]:
        """
        Configure the next round of training. Samples clients for 
        training and defines the configurations for them. This
        method is called by the server.
        """

        config = {}
        fit_ins = FitIns(parameters, config)

        # Sample clients
        sample_size, min_num_clients = self.num_fit_clients(
            client_manager.num_available()
        )
        clients = client_manager.sample(
            num_clients=sample_size, min_num_clients=min_num_clients
        )

        fit_configurations = [(client, fit_ins) for client in clients]
        
        return fit_configurations


    def aggregate_fit(
        self, 
        server_round: int, 
        results: List[Tuple[ClientProxy, FitRes]],
        failures: List[Union[Tuple[ClientProxy, FitRes], BaseException]],
    ) -> Tuple[Optional[Parameters], Dict[str, Scalar]]:
        """
        aggregation method for Scaffold strategy. Due to how 
        the ScaffoldClient returns the parameters for the global
        update, we can reuse the "aggregate" function from 
        flwr.server.strategy.aggregate (FedAvg aggregation).
        """
        if not results:
            return None, {}
        if not self.accept_failures and failures:
            return None, {}
        
        combined_parameters = [
            parameters_to_ndarrays(fit_res.parameters) for _, fit_res in results
        ]

        num_samples_all = [fit_res.num_examples for _, fit_res in results]  # number of training samples from each client

        aggregation_inputs = [
            (local_params, num_samples)
            for local_params, num_samples in zip(combined_parameters, num_samples_all)
        ]
        aggregated_parameters = aggregate(aggregation_inputs)

        metrics_aggregated = {}
        if self.fit_metrics_aggregation_fn is not None:
            fit_metrics = [
                (fit_res.num_examples, fit_res.metrics)
                for _, fit_res in results
            ]
            metrics_aggregated = self.fit_metrics_aggregation_fn(fit_metrics)
        elif server_round == 1:
            log(WARNING, "No fit_metrics_aggregation_fn provided")
        
        return (
            ndarrays_to_parameters(aggregated_parameters),
            metrics_aggregated,
        )


    def configure_evaluate(
        self, 
        server_round: int, 
        parameters: Parameters, 
        client_manager: ClientManager
    ) -> List[Tuple[ClientProxy, EvaluateIns]]:
        """Configure the next round of evaluation."""
        if self.fraction_evaluate == 0.0:
            return []
        config = {}
        evaluate_ins = EvaluateIns(parameters, config)

        # Sample clients
        sample_size, min_num_clients = self.num_evaluation_clients(
            client_manager.num_available()
        )
        clients = client_manager.sample(
            num_clients=sample_size, min_num_clients=min_num_clients
        )

        # Return client/config pairs
        return [(client, evaluate_ins) for client in clients]


    def aggregate_evaluate(
        self,
        server_round: int,
        results: List[Tuple[ClientProxy, EvaluateRes]],
        failures: List[Union[Tuple[ClientProxy, EvaluateRes], BaseException]],
    ) -> Tuple[Optional[float], Dict[str, Scalar]]:
        """Aggregate evaluation losses using weighted average."""

        if not results:
            return None, {}

        loss_aggregated = weighted_loss_avg(
            [
                (evaluate_res.num_examples, evaluate_res.loss)
                for _, evaluate_res in results
            ]
        )
        metrics_aggregated = {}
        return loss_aggregated, metrics_aggregated


    # method for evaluating the global model
    def evaluate(
            self, 
            server_round: int, 
            parameters: Parameters
        ) -> Optional[Tuple[float, Dict[str, Scalar]]]:
        """
        Evaluate global model parameters using an evaluation function.
        """
        if self.evaluate_fn is None:
            return None
            # If an evaluation function is provided, use it
        parameters_ndarray = parameters_to_ndarrays(parameters)
        eval_res = self.evaluate_fn(server_round, parameters_ndarray, {})
        if eval_res is None:
            return None
        loss, metrics = eval_res
        return loss, metrics

Finally, we are ready to define the server.

In [12]:
def concat_params(
        parameters: Parameters, 
        global_cv: List[np.ndarray]
    ) -> Parameters:
    """
    Concatenate model parameters and global control variates.
    """
    parameters_ndarrays = parameters_to_ndarrays(parameters)
    parameters_ndarrays.extend(global_cv)
    return ndarrays_to_parameters(parameters_ndarrays)


class ScaffoldServer(Server):

    def __init__(
            self, 
            strategy: Strategy, 
            client_manager: Optional[ClientManager] = None,
        ) -> None:
        
        if client_manager is None:
            client_manager = SimpleClientManager()

        super().__init__(strategy=strategy, client_manager=client_manager)
        
        self.global_cv: List[np.ndarray] = []  # Global control variates for Scaffold
    
    def _get_initial_parameters(
            self, 
            server_round: int, 
            timeout: Optional[float]
        ) -> Parameters: 
        
        parameters: Optional[Parameters] = self.strategy.initialize_parameters(
            self.client_manager
        )
    
        if parameters is not None:
            log(INFO, "Using initial parameters provided by strategy")
        else:
            # Get initial parameters from one of the clients
            log(INFO, "Requesting initial parameters from one random client")
            random_client = self._client_manager.sample(1)[0]
            ins = GetParametersIns(config={})
            get_parameters_res = random_client.get_parameters(
                ins=ins, timeout=timeout, group_id=server_round
            )
            if get_parameters_res.status.code == Code.OK:
                log(INFO, "Received initial parameters from one random client")
            else:
                log(
                    WARN,
                    "Failed to receive initial parameters from the client."
                    " Empty initial parameters will be used.",
                )
            parameters = get_parameters_res.parameters
            
        self.global_cv = [
                np.zeros_like(param, dtype=np.float32) for param in parameters_to_ndarrays(parameters)
            ]
        
        return parameters

        


    def fit_round(
            self,
            server_round: int,
            timeout: Optional[float],
        ) -> Optional[Tuple[Optional[Parameters], Dict[str, Scalar], FitResultsAndFailures]]:
        
        # define client instructions to be passed to "fit_clients" function
        client_instructions = self.strategy.configure_fit(
            server_round=server_round,
            parameters=concat_params(self.parameters, self.global_cv),
            client_manager=self._client_manager,
        )

        # if no clients are selected, return None
        if not client_instructions:
            log(INFO, f"fit_round {server_round}: no clients selected.")
            return None
        
        log(
            DEBUG,
            f"fit_round {server_round}: selected {len(client_instructions)} clients.",
        )

        # Call the "fit_clients" function to perform the training on selected clients
        results, failures = fit_clients(
            client_instructions=client_instructions,
            max_workers=self.max_workers,
            timeout=timeout,
            group_id=server_round,
        )

        log(
            DEBUG,
            f"fit_round {server_round}: received {len(results)} results and {len(failures)} failures.",
        )

        # Aggregate the results from the clients
        aggregated_results = self.strategy.aggregate_fit(
            server_round=server_round,
            results=results,
            failures=failures,
        )

        # Extract the aggregated parameters and control variates
        aggregated_results_combined = []
        if aggregated_results[0] is not None:
            aggregated_results_combined = parameters_to_ndarrays(aggregated_results[0])

        # Split the aggregated results into model parameters and control variates
        aggregated_parameters = aggregated_results_combined[:len(aggregated_results_combined) // 2] # model parameters
        aggregated_cv = aggregated_results_combined[len(aggregated_results_combined) // 2:]         # control variates

        # define the update coefficient for the control variates
        cv_coeff = len(results) / len(self._client_manager.all())

        # Update the global control variates according to
        # global_cv <- global_cv + cv_coeff * aggregated_cv
        # where cv_coeff = |S| / N, |S| is the number of clients that participated in the round
        # and aggregated_cv = (1 / |S|) * sum_{i in S} (c_i^+ - c_i)
        self.global_cv = [
            cv + cv_coeff * new_cv for cv, new_cv in zip(self.global_cv, aggregated_cv)
        ]


        # Update the global model parameters
        # new_parameters = current_parameters + aggregated_parameters
        # where current_parameters are the parameters of the global model before the round
        # and aggregated_parameters = (1 / |S|) * sum_{i in S} (w_i^+ - w)
        current_parameters = parameters_to_ndarrays(self.parameters)
        new_parameters = [
            param + update for param, update in zip(current_parameters, aggregated_parameters)
        ]

        new_parameters = ndarrays_to_parameters(new_parameters)

        return new_parameters, aggregated_results[1], (results, failures)

We are now ready for the server

In [13]:
# Create an instance of the model and get the parameters
params = get_parameters(SmallCNN())
criterion = nn.CrossEntropyLoss()

# The `evaluate` function will be called by Flower after every round
def evaluate(
        server_round: int,
        parameters: list[np.ndarray],
        config: dict[str, Scalar],
    ) -> Optional[Tuple[float, dict[str, Scalar]]]:
    
    net = SmallCNN().to(DEVICE)
    _, _, testloader = load_datasets(   
        partition_id=0,                         # specify partition ID to load
        partition_method=PARTITION_METHOD,      # Method to partition the dataset
        partitioner_kwargs=PARTITIONER_KWARGS,  # Parameters for the partitioner
        batch_size=BATCH_SIZE,                  # Batch size for the DataLoader    
        cache_dir=DATADIR                       # Directory to cache the datasets
    )
    set_parameters(net, parameters)  # Update model with the latest parameters
    loss, accuracy = test(
        net=net, 
        device=DEVICE, 
        testloader=testloader, 
        criterion=criterion,
        )
    print(f"Server-side evaluation loss {loss} / accuracy {accuracy}")
    return loss, {"accuracy": accuracy}



def server_fn(context: Context) -> ServerAppComponents:
    # Create FedAvg strategy
    #strategy = ScaffoldStrategy(
    #    total_num_clients=NUM_PARTITIONS,       # Total number of clients
    #    fraction_fit=1.0,                       # Use all clients for training, C
    #    fraction_evaluate=0.5,                  # Use 50% of clients for evaluation
    #    min_fit_clients=10,                     # Minimum number of clients to train
    #    min_evaluate_clients=5,                 # Minimum number of clients to evaluate
    #    min_available_clients=NUM_PARTITIONS,   # Minimum number of clients available (enforce all clients to be available)
    #    evaluate_fn=evaluate,                   # Pass the evaluation function
    #)
    strategy = FedAvg(
        fraction_fit=1.0,                       # Use all clients for training, C
        fraction_evaluate=0.5,                  # Use 50% of clients for evaluation
        min_fit_clients=10,                     # Minimum number of clients to train
        min_evaluate_clients=5,                 # Minimum number of clients to evaluate
        min_available_clients=NUM_PARTITIONS,   # Minimum number of clients available (enforce all clients to be available)
        evaluate_fn=evaluate,                   # Pass the evaluation function
    )

    server = ScaffoldServer(strategy=strategy)

    # Configure the server for 3 rounds of training
    config = ServerConfig(num_rounds=2)
    return ServerAppComponents(server=server, config=config)

# Create the ServerApp
server = ServerApp(server_fn=server_fn)

In [14]:
NUM_PARTITIONS = 10  # Number of partitions (clients)
backend_config = {
    #"ray_init_args": {
    #    "num_cpus": 1,
    #    "num_gpus": 1,
    #},
    "client_resources": {
        "num_cpus": 2,
        "num_gpus": 0.2,
    }
}
run_simulation(
    server_app=server, client_app=client, num_supernodes=NUM_PARTITIONS, backend_config=backend_config,
)

INFO :      Starting Flower ServerApp, config: num_rounds=2, no round_timeout
INFO :      


INFO :      [INIT]
INFO :      Requesting initial parameters from one random client
(ClientAppActor pid=27680) Using the latest cached version of the dataset since cifar10 couldn't be found on the Hugging Face Hub
(ClientAppActor pid=27680) Found the latest cached dataset configuration 'plain_text' at c:\Users\Phill\Documents\School\MSc\3. Sem\Federated Learning\FedLearning-Final\data\flower_dataset\cifar10\plain_text\0.0.0\0b2714987fa478483af9968de7c934580d0bb9a2 (last modified on Wed Jun  4 13:48:33 2025).
INFO :      Received initial parameters from one random client
INFO :      Starting evaluation of initial global parameters
INFO :      initial parameters (loss, other metrics): 2.3030722991676087, {'accuracy': 0.0991}
INFO :      
INFO :      [ROUND 1]


Server-side evaluation loss 2.3030722991676087 / accuracy 0.0991


(ClientAppActor pid=34104) Using the latest cached version of the dataset since cifar10 couldn't be found on the Hugging Face Hub
(ClientAppActor pid=34104) Found the latest cached dataset configuration 'plain_text' at c:\Users\Phill\Documents\School\MSc\3. Sem\Federated Learning\FedLearning-Final\data\flower_dataset\cifar10\plain_text\0.0.0\0b2714987fa478483af9968de7c934580d0bb9a2 (last modified on Wed Jun  4 13:48:33 2025).
(ClientAppActor pid=31516) Using the latest cached version of the dataset since cifar10 couldn't be found on the Hugging Face Hub
(ClientAppActor pid=31516) Found the latest cached dataset configuration 'plain_text' at c:\Users\Phill\Documents\School\MSc\3. Sem\Federated Learning\FedLearning-Final\data\flower_dataset\cifar10\plain_text\0.0.0\0b2714987fa478483af9968de7c934580d0bb9a2 (last modified on Wed Jun  4 13:48:33 2025).
INFO :      fit progress: (1, 2.2482746421911153, {'accuracy': 0.1719}, 83.19535709999036)
INFO :      configure_evaluate: strategy sampled 

Server-side evaluation loss 2.2482746421911153 / accuracy 0.1719


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [ROUND 2]
INFO :      fit progress: (2, 2.046463131904602, {'accuracy': 0.2076}, 149.09314500004984)
INFO :      configure_evaluate: strategy sampled 5 clients (out of 10)


Server-side evaluation loss 2.046463131904602 / accuracy 0.2076


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 2 round(s) in 150.79s
INFO :      	History (loss, distributed):
INFO :      		round 1: 2.2540293767534454
INFO :      		round 2: 1.9709022027604721
INFO :      	History (loss, centralized):
INFO :      		round 0: 2.3030722991676087
INFO :      		round 1: 2.2482746421911153
INFO :      		round 2: 2.046463131904602
INFO :      	History (metrics, centralized):
INFO :      	{'accuracy': [(0, 0.0991), (1, 0.1719), (2, 0.2076)]}
INFO :      
(ClientAppActor pid=26908) Using the latest cached version of the dataset since cifar10 couldn't be found on the Hugging Face Hub [repeated 2x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#log-deduplication for more options.)
(ClientAppActor pid=26908) Found the latest cached dataset con